In [7]:
def xor():
    global y
    y = y ^ (y << 13)
    y = y ^ (y >> 17)
    y = y ^ (y << 5)
    return y

def xor64():
    global x
    x = x ^ (x << 13)
    x = x ^ (x >> 7)
    x = x ^ (x << 17)
    return x

def xor96():
    global x96, y96, z96
    t = (x96 ^ (x96 << 3)) ^ (y96 ^ (y96 >> 19)) ^ (z96 ^ (z96 << 6))
    x96 = y96
    y96 = z96
    return z96 ^ (z96 << 16) ^ t

def xor128():
    global x128, y128, z128, w128
    t = x128 ^ (x128 << 11)
    x128 = y128
    y128 = z128
    z128 = w128
    w128 = (w128 ^ (w128 >> 19)) ^ (t ^ (t >> 8))
    return w128

# 初期値設定
y = 2463534242
x = 88172645463325252
x96 = 123456789
y96 = 362436069
z96 = 521288629
x128 = 123456789
y128 = 362436069
z128 = 521288629
w128 = 88675123

# テスト
print("xor:", xor())
print("xor64:", xor64())
print("xor96:", xor96())
print("xor128:", xor128())


xor: 660888219700579
xor64: 95272899234400648529057200
xor96: 34178664243115
xor128: 252977563114


提供されたコードは、異なるビット長のXORSHIFT乱数生成アルゴリズムをPythonで実装しています。各関数は、与えられた状態に基づいて疑似乱数を生成します。以下に、各関数の概要と動作を詳しく説明します。

1. `xor()`:
   - 32ビットのXORSHIFTアルゴリズムを実装しています。
   - グローバル変数 `y` を状態として使用し、その値を変換して新しい状態を生成します。
   - 左シフト (`<<`) と右シフト (`>>`) 、XOR演算 (`^`) を組み合わせて、状態を変更します。
   - 最後に、変更された状態 `y` を返します。

2. `xor64()`:
   - 64ビットのXORSHIFTアルゴリズムを実装しています。
   - グローバル変数 `x` を状態として使用し、同様に変換して新しい状態を生成します。
   - 左シフト (`<<`) と右シフト (`>>`) 、XOR演算 (`^`) を組み合わせて、状態を変更します。
   - 最後に、変更された状態 `x` を返します。

3. `xor96()`:
   - 96ビットのXORSHIFTアルゴリズムを実装しています。
   - グローバル変数 `x96`, `y96`, `z96` を状態として使用し、同様に変換して新しい状態を生成します。
   - 左シフト (`<<`) と右シフト (`>>`) 、XOR演算 (`^`) を組み合わせて、状態を変更します。
   - 最後に、変更された状態 `z96` を返します。

4. `xor128()`:
   - 128ビットのXORSHIFTアルゴリズムを実装しています。
   - グローバル変数 `x128`, `y128`, `z128`, `w128` を状態として使用し、同様に変換して新しい状態を生成します。
   - 左シフト (`<<`) と右シフト (`>>`) 、XOR演算 (`^`) を組み合わせて、状態を変更します。
   - 最後に、変更された状態 `w128` を返します。

これらの関数は、各状態が前回の呼び出しで変更され、その変更された状態に基づいて新しい状態が生成されることに注意してください。そして、それぞれの関数は、それぞれのビット長に基づいて異なるシフト量やXOR演算を使用しています。

In [8]:
"""
implement xorshift with python
https://ja.wikipedia.org/wiki/Xorshift
https://qiita.com/yosgspec/items/e4287262f8dbea2aa815
https://ask.helplib.com/1591921
"""

def xorshift(generator, seed=None):
    ret = seed
    def inner():
        nonlocal ret
        if ret is None:
            ret = generator()
        else:
            ret = generator(*ret)
        return ret[-1]
    return inner


def xor32(y=2463534242):
    y = y ^ (y << 13 & 0xFFFFFFFF)
    y = y ^ (y >> 17 & 0xFFFFFFFF)
    y = y ^ (y << 5 & 0xFFFFFFFF)
    return y & 0xFFFFFFFF,


#def xor64(_x=88172645463325252, x=88172645463325252):
#    _x = _x ^ (_x << 13)
#    _x = _x ^ (_x >> 7)
#    _x = _x ^ (_x << 17)
#    return _x, _x & 0xFFFFFFFF


def xor96(x=123456789, y=362436069, z=521288629):
    t = (x ^ (x << 3 & 0xFFFFFFFF)) ^ (
        y ^ (y >> 19 & 0xFFFFFFFF)) ^ (
        z ^ (z << 6 & 0xFFFFFFFF))
    x = y
    y = z
    z = t
    return x, y, z


def xor128(x=123456789, y=362436069, z=521288629, w=88675123):
    t = x ^ (x << 11) & 0xFFFFFFFF
    x = y
    y = z
    z = w
    w = (w ^ (w >> 19)) ^ (t ^ (t >> 8)) & 0xFFFFFFFF
    return x, y, z, w


def uniform(rand, begin=0, end=1):
    def inner():
        return begin+(rand())/(int(0xFFFFFFFF)/(end-begin))
    return inner


def calc_pi(generator):
    u01 = uniform(generator)
    counter = 0
    N = 100000
    for i in range(N):
        x = u01()
        y = u01()
        if x*x+y*y < 1.0:
            counter += 1
    print(4.0*counter/N)


def apply_example():
    random32 = xorshift(xor32)
    calc_pi(random32)
    #random64 = xorshift(xor64)
    #calc_pi(random64)
    random96 = xorshift(xor96)
    calc_pi(random96)
    random128 = xorshift(xor128)
    calc_pi(random128)

def main():
    random32=xorshift(xor32)
    for i in range(100):
        print(random32())

    apply_example()
if __name__ == '__main__':
    main()


723471715
2497366906
2064144800
2008045182
3532304609
374114282
1350636274
691148861
746858951
2653896249
1156348781
3149294349
2888432806
3826506360
1959669526
2495235968
1427053829
1666395154
3707535418
3548851879
4230571086
3300478942
1159583391
101148280
3016388764
1189625968
2452260707
2585150976
136020491
1929452262
1541647304
4258081829
1367187489
1246208317
1919655853
844747554
261058748
3532164720
2879067571
3968808325
1305314266
1264655649
1885610275
2085051235
1917152822
805043991
1396692091
2678912195
665198457
1835067133
1179695714
1921421164
3456140793
155623014
2345424707
631426545
4287015967
1789819581
449347681
1380975971
225824701
616860284
81872793
2925282192
3823386727
385492979
311013585
3068383640
1455835632
1626916748
2032536200
3063775443
3598422416
762426676
1630544703
3484009236
246163153
3300802218
1845796095
1325089906
3906469448
4106243367
1375641909
1053833465
3114974
3931359540
4258599246
2690853040
1809582922
3155360784
2441164504
1545061333
701173123
11